Connecting with Gdrive to read data

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#if data is in Zip format then it can be unzip using this 

#!unzip "/content/drive/My Drive/my_data_zip_file.zip"

In [3]:
#Spark is written in the Scala programming language and requires the Java Virtual Machine (JVM) to run.
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [13]:
#install Apache Spark  with Hadoop 
!wget -q https://archive.apache.org/dist/spark/spark-3.0.0/spark-3.0.0-bin-hadoop3.2.tgz


In [14]:
#need to unzip that folder.
!tar xf spark-3.0.0-bin-hadoop3.2.tgz

In [15]:
# set your spark folder to your system path environment. 
#This will enable us to run Pyspark in the Colab environment
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.0.0-bin-hadoop3.2"

In [16]:
#It will locate Spark on the system and import it as a regular library.
!pip install -q findspark

In [17]:
import findspark
findspark.init()

In [18]:
 #If you want to know the location where Spark is installed, use findspark.find()
 findspark.find()


'/content/spark-3.0.0-bin-hadoop3.2'

In [35]:
#Now, we can import SparkSession from pyspark.sql and create a SparkSession, which is the entry point to Spark.
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [20]:
spark  #printing sparksession variable

In [22]:
#if you want to view the Spark UI
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip
get_ipython().system_raw('./ngrok http 4050 &')
!curl -s http://localhost:4040/api/tunnels

--2022-06-16 05:10:58--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 54.237.133.81, 52.202.168.65, 54.161.241.46, ...
Connecting to bin.equinox.io (bin.equinox.io)|54.237.133.81|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip.1’

ngrok-stable-linux- 100%[===================>]  13.19M  4.49MB/s    in 2.9s    

2022-06-16 05:11:01 (4.49 MB/s) - ‘ngrok-stable-linux-amd64.zip.1’ saved [13832437/13832437]

Archive:  ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: ngrok                   
{"tunnels":[{"name":"command_line","uri":"/api/tunnels/command_line","public_url":"https://320e-107-167-180-196.ngrok.io","proto":"https","config":{"addr":"http://localhost:4050","inspect":true},"metrics":{"conns":{"count":0,"gauge":0,"rate1":0,"rate5":0,"rate15":0,"p50":0,"p90":0,"p95"

In [120]:
#df = spark.read.format("text").load("/content/sample_data/example_log.log",sep="|")
df = spark.read.csv("/content/sample_data/example_log.log",sep="|")

In [121]:
df.show()

+--------------------+-----------+--------------------+---------+---+--------------------+
|                 _c0|        _c1|                 _c2|      _c3|_c4|                 _c5|
+--------------------+-----------+--------------------+---------+---+--------------------+
|<166>Jun  1 07:33...|Trend Micro|Deep Security Man...|50.0.1189|851|cat=System#011nam...|
|<166>Jun  1 07:39...|Trend Micro|Deep Security Man...|50.0.1189|851|cat=System#011nam...|
|<166>Jun  1 07:44...|Trend Micro|Deep Security Man...|50.0.1189|851|cat=System#011nam...|
|<166>Jun  1 07:50...|Trend Micro|Deep Security Man...|50.0.1189|851|cat=System#011nam...|
|<166>Jun  1 07:56...|Trend Micro|Deep Security Man...|50.0.1189|851|cat=System#011nam...|
|<166>Jun  1 08:01...|Trend Micro|Deep Security Man...|50.0.1189|851|cat=System#011nam...|
|<166>Jun  1 08:07...|Trend Micro|Deep Security Man...|50.0.1189|851|cat=System#011nam...|
|<166>Jun  1 08:13...|Trend Micro|Deep Security Man...|50.0.1189|851|cat=System#011nam...|

In [122]:
import pandas as pd
df.limit(5).toPandas()

,_c0,_c1,_c2,_c3,_c4,_c5
0,<166>Jun 1 07:33:21 TM-JIO-Console-1 LEEF: 2.0,Trend Micro,Deep Security Manager,50.0.1189,851,cat=System#011name=Reconnaissance Detected: Ne...
1,<166>Jun 1 07:39:01 TM-JIO-Console-1 LEEF: 2.0,Trend Micro,Deep Security Manager,50.0.1189,851,cat=System#011name=Reconnaissance Detected: Ne...
2,<166>Jun 1 07:44:42 TM-JIO-Console-1 LEEF: 2.0,Trend Micro,Deep Security Manager,50.0.1189,851,cat=System#011name=Reconnaissance Detected: Ne...
3,<166>Jun 1 07:50:23 TM-JIO-Console-1 LEEF: 2.0,Trend Micro,Deep Security Manager,50.0.1189,851,cat=System#011name=Reconnaissance Detected: Ne...
4,<166>Jun 1 07:56:04 TM-JIO-Console-1 LEEF: 2.0,Trend Micro,Deep Security Manager,50.0.1189,851,cat=System#011name=Reconnaissance Detected: Ne...


In [141]:
import pyspark.sql.functions as F
df1=df.withColumn("_c6",F.regexp_extract(df._c0,'(>[a-zA-Z]+\s*\d+\s*[\d+:]+)',1))

In [142]:
df1.show()

+--------------------+-----------+--------------------+---------+---+--------------------+----------------+
|                 _c0|        _c1|                 _c2|      _c3|_c4|                 _c5|             _c6|
+--------------------+-----------+--------------------+---------+---+--------------------+----------------+
|<166>Jun  1 07:33...|Trend Micro|Deep Security Man...|50.0.1189|851|cat=System#011nam...|>Jun  1 07:33:21|
|<166>Jun  1 07:39...|Trend Micro|Deep Security Man...|50.0.1189|851|cat=System#011nam...|>Jun  1 07:39:01|
|<166>Jun  1 07:44...|Trend Micro|Deep Security Man...|50.0.1189|851|cat=System#011nam...|>Jun  1 07:44:42|
|<166>Jun  1 07:50...|Trend Micro|Deep Security Man...|50.0.1189|851|cat=System#011nam...|>Jun  1 07:50:23|
|<166>Jun  1 07:56...|Trend Micro|Deep Security Man...|50.0.1189|851|cat=System#011nam...|>Jun  1 07:56:04|
|<166>Jun  1 08:01...|Trend Micro|Deep Security Man...|50.0.1189|851|cat=System#011nam...|>Jun  1 08:01:45|
|<166>Jun  1 08:07...|Trend 

In [143]:
df1=df1.withColumn("Time",F.regexp_extract(df1._c6,'([\d+:]+\d)',1))

In [144]:

df1.show()

+--------------------+-----------+--------------------+---------+---+--------------------+----------------+--------+
|                 _c0|        _c1|                 _c2|      _c3|_c4|                 _c5|             _c6|    Time|
+--------------------+-----------+--------------------+---------+---+--------------------+----------------+--------+
|<166>Jun  1 07:33...|Trend Micro|Deep Security Man...|50.0.1189|851|cat=System#011nam...|>Jun  1 07:33:21|07:33:21|
|<166>Jun  1 07:39...|Trend Micro|Deep Security Man...|50.0.1189|851|cat=System#011nam...|>Jun  1 07:39:01|07:39:01|
|<166>Jun  1 07:44...|Trend Micro|Deep Security Man...|50.0.1189|851|cat=System#011nam...|>Jun  1 07:44:42|07:44:42|
|<166>Jun  1 07:50...|Trend Micro|Deep Security Man...|50.0.1189|851|cat=System#011nam...|>Jun  1 07:50:23|07:50:23|
|<166>Jun  1 07:56...|Trend Micro|Deep Security Man...|50.0.1189|851|cat=System#011nam...|>Jun  1 07:56:04|07:56:04|
|<166>Jun  1 08:01...|Trend Micro|Deep Security Man...|50.0.1189

In [145]:
df1=df1.withColumn('_c6', F.regexp_replace('_c6', '>', '')).withColumn('Date', F.regexp_replace('_c6', '([\d+:]+\d)', '')).withColumn('Data_source_host_name', F.regexp_extract(df1._c0, '([a-zA-Z]+[-][a-zA-Z]+[-][a-zA-Z]+[-]\d+)',1))

df1.show()

+--------------------+-----------+--------------------+---------+---+--------------------+---------------+--------+-------+---------------------+
|                 _c0|        _c1|                 _c2|      _c3|_c4|                 _c5|            _c6|    Time|   Date|Data_source_host_name|
+--------------------+-----------+--------------------+---------+---+--------------------+---------------+--------+-------+---------------------+
|<166>Jun  1 07:33...|Trend Micro|Deep Security Man...|50.0.1189|851|cat=System#011nam...|Jun  1 07:33:21|07:33:21|Jun  1 |     TM-JIO-Console-1|
|<166>Jun  1 07:39...|Trend Micro|Deep Security Man...|50.0.1189|851|cat=System#011nam...|Jun  1 07:39:01|07:39:01|Jun  1 |     TM-JIO-Console-1|
|<166>Jun  1 07:44...|Trend Micro|Deep Security Man...|50.0.1189|851|cat=System#011nam...|Jun  1 07:44:42|07:44:42|Jun  1 |     TM-JIO-Console-1|
|<166>Jun  1 07:50...|Trend Micro|Deep Security Man...|50.0.1189|851|cat=System#011nam...|Jun  1 07:50:23|07:50:23|Jun  1 | 

In [146]:
df1=df1.withColumn('log_category',F.regexp_replace(F.regexp_extract(df1._c5,'(cat=[a-zA-Z]+)',1),'cat=',""))
df1.show()

+--------------------+-----------+--------------------+---------+---+--------------------+---------------+--------+-------+---------------------+------------+
|                 _c0|        _c1|                 _c2|      _c3|_c4|                 _c5|            _c6|    Time|   Date|Data_source_host_name|log_category|
+--------------------+-----------+--------------------+---------+---+--------------------+---------------+--------+-------+---------------------+------------+
|<166>Jun  1 07:33...|Trend Micro|Deep Security Man...|50.0.1189|851|cat=System#011nam...|Jun  1 07:33:21|07:33:21|Jun  1 |     TM-JIO-Console-1|      System|
|<166>Jun  1 07:39...|Trend Micro|Deep Security Man...|50.0.1189|851|cat=System#011nam...|Jun  1 07:39:01|07:39:01|Jun  1 |     TM-JIO-Console-1|      System|
|<166>Jun  1 07:44...|Trend Micro|Deep Security Man...|50.0.1189|851|cat=System#011nam...|Jun  1 07:44:42|07:44:42|Jun  1 |     TM-JIO-Console-1|      System|
|<166>Jun  1 07:50...|Trend Micro|Deep Securit

In [147]:
df1=df1.withColumn("attack_type",F.regexp_replace(F.regexp_extract(df1._c5,'(#011name=[a-zA-Z:\s]+)',1),'#011name=',""))
df1.show()

+--------------------+-----------+--------------------+---------+---+--------------------+---------------+--------+-------+---------------------+------------+--------------------+
|                 _c0|        _c1|                 _c2|      _c3|_c4|                 _c5|            _c6|    Time|   Date|Data_source_host_name|log_category|         attack_type|
+--------------------+-----------+--------------------+---------+---+--------------------+---------------+--------+-------+---------------------+------------+--------------------+
|<166>Jun  1 07:33...|Trend Micro|Deep Security Man...|50.0.1189|851|cat=System#011nam...|Jun  1 07:33:21|07:33:21|Jun  1 |     TM-JIO-Console-1|      System|Reconnaissance De...|
|<166>Jun  1 07:39...|Trend Micro|Deep Security Man...|50.0.1189|851|cat=System#011nam...|Jun  1 07:39:01|07:39:01|Jun  1 |     TM-JIO-Console-1|      System|Reconnaissance De...|
|<166>Jun  1 07:44...|Trend Micro|Deep Security Man...|50.0.1189|851|cat=System#011nam...|Jun  1 07:

In [148]:
df1=df1.withColumn("description",F.regexp_replace(F.regexp_extract(df1._c5,'(011desc=[a-zA-Z\s\/.]+)',1),'011desc=',""))
df1.show()

+--------------------+-----------+--------------------+---------+---+--------------------+---------------+--------+-------+---------------------+------------+--------------------+--------------------+
|                 _c0|        _c1|                 _c2|      _c3|_c4|                 _c5|            _c6|    Time|   Date|Data_source_host_name|log_category|         attack_type|         description|
+--------------------+-----------+--------------------+---------+---+--------------------+---------------+--------+-------+---------------------+------------+--------------------+--------------------+
|<166>Jun  1 07:33...|Trend Micro|Deep Security Man...|50.0.1189|851|cat=System#011nam...|Jun  1 07:33:21|07:33:21|Jun  1 |     TM-JIO-Console-1|      System|Reconnaissance De...|The Agent/Applian...|
|<166>Jun  1 07:39...|Trend Micro|Deep Security Man...|50.0.1189|851|cat=System#011nam...|Jun  1 07:39:01|07:39:01|Jun  1 |     TM-JIO-Console-1|      System|Reconnaissance De...|The Agent/Applian

In [149]:
df1=df1.withColumn("seviority",F.regexp_replace(F.regexp_extract(df1._c5,'(011sev=[\d]+)',1),'011sev=',""))
df1.show()

+--------------------+-----------+--------------------+---------+---+--------------------+---------------+--------+-------+---------------------+------------+--------------------+--------------------+---------+
|                 _c0|        _c1|                 _c2|      _c3|_c4|                 _c5|            _c6|    Time|   Date|Data_source_host_name|log_category|         attack_type|         description|seviority|
+--------------------+-----------+--------------------+---------+---+--------------------+---------------+--------+-------+---------------------+------------+--------------------+--------------------+---------+
|<166>Jun  1 07:33...|Trend Micro|Deep Security Man...|50.0.1189|851|cat=System#011nam...|Jun  1 07:33:21|07:33:21|Jun  1 |     TM-JIO-Console-1|      System|Reconnaissance De...|The Agent/Applian...|        6|
|<166>Jun  1 07:39...|Trend Micro|Deep Security Man...|50.0.1189|851|cat=System#011nam...|Jun  1 07:39:01|07:39:01|Jun  1 |     TM-JIO-Console-1|      Syste

In [151]:
df1=df1.withColumn("src_ip",F.regexp_replace(F.regexp_extract(df1._c5,'(#011src=[0-9]{1,3}[.][0-9]{1,3}[.][0-9]{1,3}[.][0-9]{1,3})',1),'#011src=',""))
df1.show()


+--------------------+-----------+--------------------+---------+---+--------------------+---------------+--------+-------+---------------------+------------+--------------------+--------------------+---------+--------------+
|                 _c0|        _c1|                 _c2|      _c3|_c4|                 _c5|            _c6|    Time|   Date|Data_source_host_name|log_category|         attack_type|         description|seviority|        src_ip|
+--------------------+-----------+--------------------+---------+---+--------------------+---------------+--------+-------+---------------------+------------+--------------------+--------------------+---------+--------------+
|<166>Jun  1 07:33...|Trend Micro|Deep Security Man...|50.0.1189|851|cat=System#011nam...|Jun  1 07:33:21|07:33:21|Jun  1 |     TM-JIO-Console-1|      System|Reconnaissance De...|The Agent/Applian...|        6|192.168.83.163|
|<166>Jun  1 07:39...|Trend Micro|Deep Security Man...|50.0.1189|851|cat=System#011nam...|Jun  1

In [159]:
df1=df1.withColumn("user_name",F.regexp_replace(F.regexp_extract(df1._c5,'(011usrName=[a-zA-Z \s]+)',1),'011usrName=',""))
df1.show()

+--------------------+-----------+--------------------+---------+---+--------------------+---------------+--------+-------+---------------------+------------+--------------------+--------------------+---------+--------------+---------+-----------------+--------------+
|                 _c0|        _c1|                 _c2|      _c3|_c4|                 _c5|            _c6|    Time|   Date|Data_source_host_name|log_category|         attack_type|         description|seviority|        src_ip|user_name|desitination_host|destination_ip|
+--------------------+-----------+--------------------+---------+---+--------------------+---------------+--------+-------+---------------------+------------+--------------------+--------------------+---------+--------------+---------+-----------------+--------------+
|<166>Jun  1 07:33...|Trend Micro|Deep Security Man...|50.0.1189|851|cat=System#011nam...|Jun  1 07:33:21|07:33:21|Jun  1 |     TM-JIO-Console-1|      System|Reconnaissance De...|The Agent/Appl

In [160]:
df1=df1.withColumn("destination_ip",F.regexp_extract(F.regexp_extract(df1._c5,'(#011target=[0-9 . ( ) a-zA-Z ]+)',1),'([0-9]{1,3}[.][0-9]{1,3}[.][0-9]{1,3}[.][0-9]{1,3})',1))
df1.show()

+--------------------+-----------+--------------------+---------+---+--------------------+---------------+--------+-------+---------------------+------------+--------------------+--------------------+---------+--------------+---------+-----------------+--------------+
|                 _c0|        _c1|                 _c2|      _c3|_c4|                 _c5|            _c6|    Time|   Date|Data_source_host_name|log_category|         attack_type|         description|seviority|        src_ip|user_name|desitination_host|destination_ip|
+--------------------+-----------+--------------------+---------+---+--------------------+---------------+--------+-------+---------------------+------------+--------------------+--------------------+---------+--------------+---------+-----------------+--------------+
|<166>Jun  1 07:33...|Trend Micro|Deep Security Man...|50.0.1189|851|cat=System#011nam...|Jun  1 07:33:21|07:33:21|Jun  1 |     TM-JIO-Console-1|      System|Reconnaissance De...|The Agent/Appl

In [158]:
df1=df1.withColumn("desitination_host",F.regexp_replace(F.regexp_replace(F.regexp_extract(df1._c5,'(#011target=[0-9 . ( ) a-zA-Z ]+)',1),'([0-9]{1,3}[.][0-9]{1,3}[.][0-9]{1,3}[.][0-9]{1,3})',""),'#011target=' ,""))
df1.show()

+--------------------+-----------+--------------------+---------+---+--------------------+---------------+--------+-------+---------------------+------------+--------------------+--------------------+---------+--------------+---------+-----------------+--------------+
|                 _c0|        _c1|                 _c2|      _c3|_c4|                 _c5|            _c6|    Time|   Date|Data_source_host_name|log_category|         attack_type|         description|seviority|        src_ip|user_name|desitination_host|destination_ip|
+--------------------+-----------+--------------------+---------+---+--------------------+---------------+--------+-------+---------------------+------------+--------------------+--------------------+---------+--------------+---------+-----------------+--------------+
|<166>Jun  1 07:33...|Trend Micro|Deep Security Man...|50.0.1189|851|cat=System#011nam...|Jun  1 07:33:21|07:33:21|Jun  1 |     TM-JIO-Console-1|      System|Reconnaissance De...|The Agent/Appl

In [161]:
df1=df1.withColumn("rule_message",F.regexp_replace(F.regexp_extract(df1._c5,'(#011msg=[0-9 . \/ a-zA-Z ]+)',1),'#011msg=',""))
df1.show()

+--------------------+-----------+--------------------+---------+---+--------------------+---------------+--------+-------+---------------------+------------+--------------------+--------------------+---------+--------------+---------+-----------------+--------------+--------------------+
|                 _c0|        _c1|                 _c2|      _c3|_c4|                 _c5|            _c6|    Time|   Date|Data_source_host_name|log_category|         attack_type|         description|seviority|        src_ip|user_name|desitination_host|destination_ip|        rule_message|
+--------------------+-----------+--------------------+---------+---+--------------------+---------------+--------+-------+---------------------+------------+--------------------+--------------------+---------+--------------+---------+-----------------+--------------+--------------------+
|<166>Jun  1 07:33...|Trend Micro|Deep Security Man...|50.0.1189|851|cat=System#011nam...|Jun  1 07:33:21|07:33:21|Jun  1 |     TM

In [162]:
df1=df1.withColumn("Data_source_ip",F.regexp_replace(F.regexp_extract(df1._c5,'(#011TrendMicroDsScannerIp=[0-9]{1,3}[.][0-9]{1,3}[.][0-9]{1,3}[.][0-9]{1,3})',1),'#011TrendMicroDsScannerIp=',""))
df1.show()

+--------------------+-----------+--------------------+---------+---+--------------------+---------------+--------+-------+---------------------+------------+--------------------+--------------------+---------+--------------+---------+-----------------+--------------+--------------------+--------------+
|                 _c0|        _c1|                 _c2|      _c3|_c4|                 _c5|            _c6|    Time|   Date|Data_source_host_name|log_category|         attack_type|         description|seviority|        src_ip|user_name|desitination_host|destination_ip|        rule_message|Data_source_ip|
+--------------------+-----------+--------------------+---------+---+--------------------+---------------+--------+-------+---------------------+------------+--------------------+--------------------+---------+--------------+---------+-----------------+--------------+--------------------+--------------+
|<166>Jun  1 07:33...|Trend Micro|Deep Security Man...|50.0.1189|851|cat=System#011na

In [163]:
df1=df1.withColumn("destination_port",F.regexp_replace(F.regexp_extract(df1._c5,'(TrendMicroDsTargetPortList=[\d*;]*\d*)',1),'TrendMicroDsTargetPortList=',""))
df1.show()

+--------------------+-----------+--------------------+---------+---+--------------------+---------------+--------+-------+---------------------+------------+--------------------+--------------------+---------+--------------+---------+-----------------+--------------+--------------------+--------------+--------------------+
|                 _c0|        _c1|                 _c2|      _c3|_c4|                 _c5|            _c6|    Time|   Date|Data_source_host_name|log_category|         attack_type|         description|seviority|        src_ip|user_name|desitination_host|destination_ip|        rule_message|Data_source_ip|    destination_port|
+--------------------+-----------+--------------------+---------+---+--------------------+---------------+--------+-------+---------------------+------------+--------------------+--------------------+---------+--------------+---------+-----------------+--------------+--------------------+--------------+--------------------+
|<166>Jun  1 07:33...|

In [168]:
df2=df1.drop('_c0','_c1','_c2','_c3','_c4','_c5','_c6')

In [169]:
df2.show()

+--------+-------+---------------------+------------+--------------------+--------------------+---------+--------------+---------+-----------------+--------------+--------------------+--------------+--------------------+
|    Time|   Date|Data_source_host_name|log_category|         attack_type|         description|seviority|        src_ip|user_name|desitination_host|destination_ip|        rule_message|Data_source_ip|    destination_port|
+--------+-------+---------------------+------------+--------------------+--------------------+---------+--------------+---------+-----------------+--------------+--------------------+--------------+--------------------+
|07:33:21|Jun  1 |     TM-JIO-Console-1|      System|Reconnaissance De...|The Agent/Applian...|        6|192.168.83.163|   System|  (sidcpaymeng08)|  10.129.76.76|The Agent/Applian...|  10.129.76.75|7001;17000;17001;...|
|07:39:01|Jun  1 |     TM-JIO-Console-1|      System|Reconnaissance De...|The Agent/Applian...|        6|192.168.83.